# LangChain Expression Language (LCEL) with Amazon Nova

This notebook demonstrates chain composition patterns using LangChain Expression Language.

## Setup

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [3]:
from langchain_amazon_nova import ChatAmazonNova
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Initialize the model
llm = ChatAmazonNova(model="nova-pro-v1", temperature=0.7, streaming=True)

## 1. Simple Chain

The most basic LCEL pattern: `prompt | model | parser`

In [4]:
# Create a simple chain
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm | StrOutputParser()

# Invoke it
result = chain.invoke({"topic": "programming"})
print(f"Result: {result}")

Result: Sure, here's a classic programming joke:

Why do programmers prefer dark mode?

Because light attracts bugs!


## 2. Sequential Chain

Chain multiple operations in sequence.

In [5]:
# First step: translate
translate_prompt = ChatPromptTemplate.from_template(
    "Translate this to {language}: {text}"
)
translate_chain = translate_prompt | llm | StrOutputParser()

# Second step: summarize
summarize_prompt = ChatPromptTemplate.from_template("Summarize in one sentence: {text}")
summarize_chain = summarize_prompt | llm | StrOutputParser()

# Test translation
translation = translate_chain.invoke(
    {"text": "LangChain is great for building AI applications", "language": "Spanish"}
)
print(f"Translation: {translation}\n")

# Then summarize the translation
summary = summarize_chain.invoke({"text": translation})
print(f"Summary: {summary}")

Translation: Here's your translation:

LangChain es ideal para crear aplicaciones de inteligencia artificial.

If you need any more translations or further assistance with anything else, feel free to ask!

Summary: LangChain is ideal for creating artificial intelligence applications. If you need more translations or further assistance, feel free to ask!


## 3. Parallel Execution

Run multiple chains in parallel using `RunnableParallel`.

In [6]:
# Define two different chains
joke_chain = (
    ChatPromptTemplate.from_template("Tell a joke about {topic}")
    | llm
    | StrOutputParser()
)

poem_chain = (
    ChatPromptTemplate.from_template("Write a haiku about {topic}")
    | llm
    | StrOutputParser()
)

# Run them in parallel
parallel_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
results = parallel_chain.invoke({"topic": "clouds"})

print("Joke:")
print(results["joke"])
print("\nPoem:")
print(results["poem"])

Joke:
Sure, here's a light-hearted joke about clouds:

---

Why did the cloud go to school?

Because it wanted to be a cumulus-nibus!

---

Hope that brought a smile to your face!

Poem:
Whispers of sky,
Clouds dance in silent grace,
Nature's haiku.


## 4. Chain with Branching Logic

Use `RunnablePassthrough` to create more complex data flows.

In [7]:
# Create a chain that analyzes and then elaborates
analyze_prompt = ChatPromptTemplate.from_template(
    "Analyze this text in 2-3 words: {text}"
)
elaborate_prompt = ChatPromptTemplate.from_template(
    "Original: {original}\nAnalysis: {analysis}\n\nElaborate on the analysis:"
)

# Chain that passes through original text while adding analysis
chain = (
    {
        "original": RunnablePassthrough(),
        "analysis": analyze_prompt | llm | StrOutputParser(),
    }
    | elaborate_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("The quick brown fox jumps over the lazy dog")
print(result)

Certainly! Let's break down the sentence "The quick brown fox jumps over the lazy dog" and elaborate on the analysis "Fox jumps."

### Sentence Breakdown

1. **Subject**: 
   - **"The quick brown fox"**
     - **"The"**: Definite article specifying a particular noun.
     - **"quick"**: Adjective describing the speed of the fox.
     - **"brown"**: Adjective describing the color of the fox.
     - **"fox"**: Noun, the main subject of the sentence.

2. **Verb**:
   - **"jumps"**: Action verb indicating the movement of the subject.

3. **Prepositional Phrase**:
   - **"over the lazy dog"**
     - **"over"**: Preposition indicating the relationship between the action and the object.
     - **"the"**: Definite article specifying a particular noun.
     - **"lazy"**: Adjective describing the state of the dog.
     - **"dog"**: Noun, the object over which the fox jumps.

### Elaborated Analysis: "Fox jumps."

1. **Subject Identification**:
   - The primary subject of the sentence is "the qui

## 5. Chain with Fallbacks

Add fallback behavior for robustness.

In [8]:
# Primary model
primary = ChatAmazonNova(model="nova-pro-v1", temperature=0.7)

# Fallback model
fallback_model = ChatAmazonNova(model="nova-lite-v1", temperature=0.7)

# Create chain with fallback
prompt = ChatPromptTemplate.from_template("What is {thing}?")
chain_with_fallback = (prompt | primary | StrOutputParser()).with_fallbacks(
    [prompt | fallback_model | StrOutputParser()]
)

result = chain_with_fallback.invoke({"thing": "LangChain"})
print(f"Result: {result}")

Result: LangChain is a framework designed to simplify the creation of applications that use large language models (LLMs). It provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications. Here’s a breakdown of its key features and components:

### Key Features:

1. **Standard Interface**:
   - LangChain offers a consistent API for interacting with different LLMs, making it easier to switch between models or use multiple models in a single application.

2. **Chains**:
   - Chains are sequences of steps where the output of one step is the input to the next. LangChain allows you to create complex workflows by chaining together various components like prompts, LLMs, and tools.

3. **Agents**:
   - Agents are more dynamic versions of chains that can make decisions about which tools to use based on the input. They are useful for tasks that require more flexibility and decision-making.

4. **Memory**:
   - LangChain provides mec

## 6. Streaming Chains

Stream output from chains token by token.

In [9]:
prompt = ChatPromptTemplate.from_template(
    "Write a short story about {topic} in 3 sentences."
)
chain = prompt | llm | StrOutputParser()

print("Streaming output:")
for chunk in chain.stream({"topic": "a robot learning to paint"}):
    print(chunk, end="", flush=True)
print("\n")

Streaming output:
In a quiet studio, a robot named Artie, programmed with algorithms of color theory and brush techniques, began to experiment with paints, guided by the delicate touch of an artist's hand. As strokes of vibrant hues graced the canvas, Artie's sensors detected a surge of emotion, a newfound appreciation for the beauty of human expression. With each painting, Artie evolved, blending logic and creativity, until its artwork whispered tales of a machine that had learned to feel through the language of art.



## 7. Chain Composition with Data Transformation

Transform data between chain steps.

In [10]:
# Function to transform data
def extract_keywords(text: str) -> dict:
    words = text.split()
    return {"keywords": ", ".join(words[:3])}


# Chain with transformation
keyword_prompt = ChatPromptTemplate.from_template(
    "Generate a title using these keywords: {keywords}"
)

chain = extract_keywords | keyword_prompt | llm | StrOutputParser()

result = chain.invoke("artificial intelligence machine learning deep neural networks")
print(f"Generated title: {result}")

Generated title: "Harnessing the Power of Artificial Intelligence: Unveiling the Future of Machine Learning"


## 8. Advanced Parameters in Chains

Use Phase 1 API parameters for fine-tuned control.

In [11]:
# Model with advanced parameters
precise_llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.3,
    max_tokens=100,
    top_p=0.9,
    reasoning_effort="high",
    metadata={"demo": "chains_notebook", "pattern": "advanced_params"},
)

creative_llm = ChatAmazonNova(
    model="nova-pro-v1", temperature=0.9, max_tokens=150, top_p=0.95
)

# Parallel chains with different parameter sets
precise_chain = (
    ChatPromptTemplate.from_template("Calculate: {problem}")
    | precise_llm
    | StrOutputParser()
)

creative_chain = (
    ChatPromptTemplate.from_template("Write a creative analogy for: {concept}")
    | creative_llm
    | StrOutputParser()
)

parallel = RunnableParallel(calculation=precise_chain, analogy=creative_chain)

results = parallel.invoke({"problem": "15% of 240", "concept": "15% of 240"})

print(f"Precise (high reasoning): {results['calculation']}")
print(f"\nCreative (high temperature): {results['analogy']}")

NovaValidationError: Error code: 400 - {'message': 'Model nova-pro-v1 does not support reasoning'}

## Summary

**LCEL Chain Patterns:**

| Pattern | Syntax | Use Case |
|---------|--------|----------|
| Simple Chain | `prompt \| llm \| parser` | Basic transformations |
| Sequential | Multiple chains in order | Multi-step processing |
| Parallel | `RunnableParallel(...)` | Independent concurrent operations |
| Branching | `RunnablePassthrough()` | Complex data flows |
| Fallbacks | `.with_fallbacks([...])` | Error handling, redundancy |
| Streaming | `.stream(...)` | Real-time output |
| Transformation | Custom functions in chain | Data preprocessing |

**Key Advantages:**
- **Composable**: Build complex workflows from simple pieces
- **Readable**: Pipeline syntax makes logic clear
- **Flexible**: Easy to add/remove/modify steps
- **Async-ready**: All chains support async execution
- **Debuggable**: Each step can be tested independently